# Transformer麻雀AI 開発ワークフロー（最終完成版）

**【目的】**
Google Driveとの連携に起因するエラーを完全に排除し、データ生成から学習までを安定して実行する。

## STEP 1: 環境設定

In [ ]:
!pip install mahjong
from google.colab import drive
drive.mount('/content/drive')

GDRIVE_PROJECT_ROOT = "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"
%cd {GDRIVE_PROJECT_ROOT}

## STEP 2: GPUの確認（最重要）

In [ ]:
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU is available.")
else:
    print("❌ GPU is NOT available. ランタイムのタイプをGPUに変更してください。")

## STEP 3: データ準備（Drive -> Colabローカル）

In [ ]:
import os
import zipfile

ZIP_FILE_PATH_ON_DRIVE = os.path.join(GDRIVE_PROJECT_ROOT, 'data.zip')
EXTRACT_DIR = '/content/data'

if not os.path.exists(EXTRACT_DIR) or not os.listdir(EXTRACT_DIR):
    print(f"Unzipping data files to local storage for fast processing...")
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    with zipfile.ZipFile(ZIP_FILE_PATH_ON_DRIVE, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR)
else:
    print("Local data directory already exists. Skipping unzip.")

!ls -l {EXTRACT_DIR} | head -n 6

## STEP 4: データセット生成（Colabローカルで完結）

**このセルを繰り返し実行してください。**
進捗はColabのローカルディスクに保存されるため、セッションが切れても安全に再開できます。

In [ ]:
!python -m src.transformer.generate_data

## STEP 5: 成果物の同期（Colabローカル -> Drive）

**STEP 4で「All files have been processed」と表示されたら、このセルを一度だけ実行してください。**
Colabのローカルに作成された全ての中間ファイルを、Google Driveにコピーして永続化します。

In [ ]:
LOCAL_PROCESSED_DIR = '/content/processed_data'
GDRIVE_PROCESSED_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')

print(f"Syncing processed data from {LOCAL_PROCESSED_DIR} to {GDRIVE_PROCESSED_DIR}...")
# rsyncを使って、中断しても再開可能な形で同期します
!rsync -ah --progress "{LOCAL_PROCESSED_DIR}/" "{GDRIVE_PROCESSED_DIR}/"
print("Sync complete!")

## STEP 6: チャンク結合 ＆ 最終データセット作成

**STEP 5の同期が完了したら、このセルを一度だけ実行してください。**
Driveに保存された中間ファイルを結合し、最終的なデータセットを作成します。

In [ ]:
import glob, pickle, numpy as np, os, tensorflow as tf
from src.transformer.vectorizer import MAX_CONTEXT_LENGTH, MAX_CHOICES

GDRIVE_PROCESSED_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')
CHUNK_DIR_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'chunks')
FINAL_DATASET_PATH_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'training_dataset_transformer.pkl')

print("--- Starting Final Merging Process ---")
chunk_files = sorted(glob.glob(os.path.join(CHUNK_DIR_GDRIVE, "*.pkl")))

if not chunk_files: print("Error: No chunk files found to merge.")
else:
    print(f"Found {len(chunk_files)} chunks. Calculating total size...")
    total_samples = sum(len(pickle.load(open(f, 'rb'))[2]) for f in chunk_files)
    print(f"Total samples to merge: {total_samples}")

    contexts_final = np.zeros((total_samples, MAX_CONTEXT_LENGTH, 100), dtype=np.float32)
    choices_final = np.zeros((total_samples, MAX_CHOICES, 100), dtype=np.float32)
    labels_final = np.zeros((total_samples,), dtype=np.int32)

    current_pos = 0
    progbar = tf.keras.utils.Progbar(len(chunk_files), unit_name="chunk")
    for i, chunk_file in enumerate(chunk_files):
        with open(chunk_file, 'rb') as f: contexts, choices, labels = pickle.load(f)
        num_in_chunk = len(labels)
        contexts_final[current_pos : current_pos + num_in_chunk] = contexts
        choices_final[current_pos : current_pos + num_in_chunk] = choices
        labels_final[current_pos : current_pos + num_in_chunk] = labels
        current_pos += num_in_chunk
        progbar.update(i + 1)

    print("\nGenerating final masks...")
    non_zero_counts = np.count_nonzero(np.sum(choices_final, axis=2), axis=1)
    masks = np.zeros_like(choices_final[:, :, 0], dtype='float32')
    for i, count in enumerate(non_zero_counts): masks[i, :count] = 1.0
    final_dataset = (contexts_final, choices_final, labels_final, masks)

    print(f"Saving final dataset to '{FINAL_DATASET_PATH_GDRIVE}'...")
    with open(FINAL_DATASET_PATH_GDRIVE, 'wb') as f: pickle.dump(final_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("\n--- Merging Complete! ---")

    print("Cleaning up temporary chunk files from Google Drive...")
    !rm -rf "{CHUNK_DIR_GDRIVE}"
    print("Done.")

## STEP 7: AIモデルの学習

In [ ]:
print("--- Starting Training ---")
!python -m src.transformer.train_transformer

print("\n--- Saving trained model to Google Drive ---")
LOCAL_MODEL_DIR = '/content/models'
GDRIVE_MODEL_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'models')
os.makedirs(GDRIVE_MODEL_DIR, exist_ok=True)
!cp -r {LOCAL_MODEL_DIR}/* "{GDRIVE_MODEL_DIR}/"

## STEP 8: 学習済みAIによる予測

In [ ]:
!python -m src.transformer.predict_transformer